# Data 

In [6]:
import pandas as pd
import numpy as np

In [113]:
df_dict={'rating':["A","B","C","D","E"],"no_of_accounts":[10,40,25,15,10],"no_of_defaults":[0,0,0,2,1]}

In [114]:
df = pd.DataFrame.from_dict(df_dict)

In [115]:
df

,rating,no_of_accounts,no_of_defaults
0,A,10,0
1,B,40,0
2,C,25,0
3,D,15,2
4,E,10,1


# Calibration

## Pluto Tasche Approach

In [1]:
def pt_one_period_pd(portfolio, defaults, c_i=0.9):
    # One Period PD estimation Pluto & Tasche model
  # Args:
  #   portf.uncond:   unconditional portfolio distribution from the worst to the best credit quality
  #   portf.def:      number of defaults in a given rating class
  #   conf.interval:  condifence interval of PD estimates
  # Returns:
  #                   estimated conditional PDs
    r_num = len(portfolio)
    r_pd = np.zeros((r_num, 1))
    portf_CNum = np.cumsum(portfolio)[::-1]
    portf_CDef = np.cumsum(defaults)[::-1]

    for r in range(r_num):

        if portf_CDef[r] == portf_CNum[r]:
            r_pd[r] = 1
        else:
            from scipy.stats import binom

            def binomial(prob):
                return binom.cdf(k=portf_CDef[r], n=portf_CNum[r], p=prob) - 1 + c_i

            from scipy.optimize import brentq

            def optimizer(f):
                return brentq(f, a=0, b=1)

            r_pd[r] = optimizer(f=binomial)

    return r_pd[::-1]

### CI = 0.9

In [2]:
pt_one_period_pd([10,15,25,40,10],[1,2,0,0,0])

array([[0.33684772],
       [0.24801831],
       [0.12875642],
       [0.07272319],
       [0.06558575]])

### CI = 0.75

In [3]:
pt_one_period_pd([10,15,25,40,10],[1,2,0,0,0],c_i=0.75)

array([[0.24737063],
       [0.19581168],
       [0.10004063],
       [0.05610747],
       [0.05055637]])

### CI = 0.5

In [4]:
pt_one_period_pd([10,15,25,40,10],[1,2,0,0,0],c_i=0.5)

array([[0.16226273],
       [0.14492462],
       [0.07294975],
       [0.0406487 ],
       [0.03659748]])

## Van Der Burght CAP method

In [20]:
def VDB_Calibrate_PD(portf_uncond, pd_uncond_old, pd_uncond_new, AR, rating_type):
    # Calibrates PDs acording to approach proposed by M. van der Burgt.
  # Args:
  #   portf_uncond:     unconditional portfolio distribution from the worst to the best credit quality
  #   pd_uncond_new:    target Mean PD (Central Tendency) for the porfolio  
  #   pd_uncond_old:    unconditional PD of the sample on wich AR had been estimated
  #   AR:               AR of the ranking model
  #   rating_type:      In case RATING, each item in the portf.uncond contains number of companies in a given rating class
  # Returns:
  #   lambda:           convexity parameter of the calibration curve
  #   pd_cond:          conditional PDs after calibration 
  #   portf_cumdist:    cumulative portfolio distribution needed to estimate logit PDs (conditional on non-default if such data is given)
  #   portf_uncond:     unconditional portfolio distribution from the worst to the best credit quality
  #   rating_type:      In case RATING, each item in the portf.uncond contains number of companies in a given rating class  
    def ecdf(data):
        """ Compute ECDF """
        from statsmodels.distributions.empirical_distribution import ECDF
        ecd = ECDF(data)
        val = [np.round(ecd(i),4) for i in data]

        return val
    
    def VDB_Get_K(): 
        # Returns:
        # approximation parameters (describes convexity of CAP curve)
        from scipy.optimize import brentq
        def optimizer(f):
            return brentq(f ,a=10**-5,b=100)

        return optimizer(f=VDBAR)
    
    def VDBAR(k):
    
        return((2 * (1 / (1 - np.exp(-k)) - 1 / k - 0.5) / (1 -  pd_uncond_old )))- AR
    
    
    def VDB_CAP_Der(x, k):
        # One parameter approximation of Derivative ofcAP curve
        # Args:
        #   k:      approximation parameters (describes convexity of CAP curve)
        #   x:      point of borrower's unconditional cumulative distribution
        # Returns:
        #         value of approximated Derivative of CAP curve in a point x
        return  ((k * np.exp(-k * x)/(1 - np.exp(-k))))
    
    from operator import add
    if rating_type == 'RATING':
        portf_cum = list(np.cumsum(portf_uncond))
        portf_cum_rl = portf_cum[:]
        portf_cum_rl.pop()
        portf_cum_rl.insert(0, 0)
        portf_dist = np.divide(
            list(map(add, portf_cum, portf_cum_rl)), (2*sum(portf_uncond)))
    else:
        portf_dist = ecdf(portf_uncond)
        portf_dist[len(portf_dist)-1] = (1+portf_dist[len(portf_dist)-2])/2

    ##############
    k = VDB_Get_K()
    ##############

    pd_cond = [pd_uncond_new * VDB_CAP_Der(i, k) for i in portf_dist]

    rez = {}
    rez["lambda"] = k
    rez['cond_PD'] = pd_cond
    rez['portf_cum_dist'] = portf_dist
    rez['rating_type'] = rating_type
    rez['portf_uncond'] = np.sum(np.multiply(np.divide(portf_uncond, sum(
        portf_uncond)), pd_cond)) if rating_type == "RATING" else np.mean(pd_cond)
    return rez

### AR=0.9

In [21]:
VDB_Calibrate_PD(portf_uncond=[10,15,25,40,10], pd_uncond_old=0.03, pd_uncond_new=0.03, AR=0.9, rating_type='RATING')

{'lambda': 15.747995588656934,
 'cond_PD': [0.21497265628673054,
  0.03002443023388319,
  0.0012871265763038152,
  7.706537436174594e-06,
  1.5032887110278232e-07],
 'portf_cum_dist': array([0.05 , 0.175, 0.375, 0.7  , 0.95 ]),
 'rating_type': 'RATING',
 'portf_uncond': 0.026325809455693065}

### AR =0.75

In [22]:
VDB_Calibrate_PD(portf_uncond=[10,15,25,40,10], pd_uncond_old=0.03, pd_uncond_new=0.03, AR=0.75, rating_type='RATING')

{'lambda': 7.303335255571316,
 'cond_PD': [0.15217562222018272,
  0.061075961274788496,
  0.014174595388102663,
  0.0013203110424987227,
  0.00021268002627655702],
 'portf_cum_dist': array([0.05 , 0.175, 0.375, 0.7  , 0.95 ]),
 'rating_type': 'RATING',
 'portf_uncond': 0.028471997679889356}

### AR =0.5

In [23]:
VDB_Calibrate_PD(portf_uncond=[10,15,25,40,10], pd_uncond_old=0.03, pd_uncond_new=0.03, AR=0.5, rating_type='RATING')

{'lambda': 3.441408057915692,
 'cond_PD': [0.08979695737730259,
  0.058403457116475016,
  0.029344157801446967,
  0.009589184558427711,
  0.004056351159784401],
 'portf_cum_dist': array([0.05 , 0.175, 0.375, 0.7  , 0.95 ]),
 'rating_type': 'RATING',
 'portf_uncond': 0.029317562694912782}